In [1]:
import numpy as np

In [1]:
import numpy as np

# Set the parameters
input_size = 2  # size of input vector
hidden_size = 10  # size of hidden layer
output_size = 1  # size of output vector
sequence_length = 1000  # length of sequence
learning_rate = 0.1  # learning rate
iterations = 5000  # number of training iterations

# Initialize the weights and biases
U = np.random.randn(hidden_size, input_size)
V = np.random.randn(output_size, hidden_size)
W = np.random.randn(hidden_size, hidden_size)
b_hidden = np.zeros((hidden_size, 1))
b_output = np.zeros((output_size, 1))

# Define the activation function (sigmoid)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the derivative of the activation function
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Generate the training data
X = np.random.randint(low=0, high=10, size=(sequence_length, input_size))
Y = np.zeros((sequence_length, output_size))
for i in range(sequence_length):
    Y[i] = np.sum(X[i])

# Train the network
for i in range(iterations):
    # Initialize the hidden state
    h = np.zeros((hidden_size, 1))

    # Forward pass
    for j in range(sequence_length):
        x = X[j].reshape((input_size, 1))
        h = sigmoid(np.dot(U, x) + np.dot(W, h) + b_hidden)
        y_hat = np.dot(V, h) + b_output

    # Calculate the loss and gradients
    loss = np.sum((y_hat - Y)**2) / sequence_length
    dy_hat = 2 * (y_hat - Y) / sequence_length
    dV = np.dot(dy_hat, h.T)
    db_output = np.sum(dy_hat, axis=1, keepdims=True)

    dU = np.zeros_like(U)
    dW = np.zeros_like(W)
    db_hidden = np.zeros_like(b_hidden)
    dh_next = np.zeros_like(h)

    # Backward pass
    for j in reversed(range(sequence_length)):
        x = X[j].reshape((input_size, 1))
        h = sigmoid(np.dot(U, x) + np.dot(W, h) + b_hidden)
        y_hat = np.dot(V, h) + b_output

        # Calculate the gradients
        dy_hat = 2 * (y_hat - Y[j]) / sequence_length
        dV += np.dot(dy_hat, h.T)
        db_output += dy_hat

        dh = np.dot(V.T, dy_hat) + dh_next
        dh_raw = sigmoid_derivative(h) * dh
        db_hidden += dh_raw
        dU += np.dot(dh_raw, x.T)
        dW += np.dot(dh_raw, h.T)
        dh_next = np.dot(W.T, dh_raw)

    # Update the weights and biases
    U -= learning_rate * dU
    V -= learning_rate * dV
    W -= learning_rate * dW
    b_hidden -= learning_rate * db_hidden
    b_output -= learning_rate * db_output

    # Print the loss every 100 iterations
    if i % 100 == 0:
        print("Iteration:", i, "Loss:", loss)


ValueError: non-broadcastable output operand with shape (1,10) doesn't match the broadcast shape (1000,10)

In [159]:
def data_gen(N, seq_len=6, high=1):
    """ A data generator for adding problem.

    The data definition strictly follows Quoc V. Le, Navdeep Jaitly, Geoffrey E.
    Hintan's paper, A Simple Way to Initialize Recurrent Networks of Rectified
    Linear Units.

    The single datum entry is a 2D vector with two rows with same length.
    The first row is a list of random data; the second row is a list of binary
    mask with all ones, except two positions sampled by uniform distribution.
    The corresponding label entry is the sum of the masked data. For
    example:

     input          label
     -----          -----
    1 4 5 3  ----->   9 (4 + 5)
    0 1 1 0

    :param N: the number of the entries.
    :param seq_len: the length of a single sequence.
    :param p: the probability of 1 in generated mask
    :param high: the random data is sampled from a [0, high] uniform distribution.
    :return: (X, Y), X the data, Y the label.
    """
    X_num = np.random.uniform(low=0, high=high, size=(N, seq_len, 1))
    X_mask = np.zeros((N, seq_len, 1))
    Y = np.ones((N, 1))
    for i in range(N):
        # Default uniform distribution on position sampling
        positions = np.random.choice(seq_len, size=2, replace=False)
        X_mask[i, positions] = 1
        X_num = np.round(X_num, 2)
        Y[i, 0] = np.round(np.sum(X_num[i, positions]), 2)
    X = np.append(X_num, X_mask, axis=2)
    return X, Y

In [162]:
data_gen(2)

(array([[[0.65, 1.  ],
         [0.23, 0.  ],
         [0.7 , 0.  ],
         [0.11, 0.  ],
         [0.31, 0.  ],
         [0.28, 1.  ]],
 
        [[0.27, 0.  ],
         [0.09, 0.  ],
         [0.8 , 0.  ],
         [0.69, 1.  ],
         [0.98, 0.  ],
         [0.1 , 1.  ]]]),
 array([[0.93],
        [0.79]]))

In [146]:
def adding_problem_generator(N):
    """ A data generator for adding problem.

    :param N: the number of the entries.
    :param seq_len: the length of a single sequence.
    :param p: the probability of 1 in generated mask
    :param high: the random data is sampled from a [0, high] uniform distribution.
    :return: (X, Y), X the data, Y the label.
    """
    X_res, Y_res = [], []
    for _ in range(N):
        seq_len = np.random.choice(range(3, 11))

        X_num = np.random.uniform(low=0, high=1, size=(1, seq_len))
        X_num = np.round(X_num, 2)
        X_mask = np.zeros((1, seq_len))
        # Default uniform distribution on position sampling
        positions = np.random.choice(seq_len, size=2, replace=False)
        X_mask[0, positions[0]] = 1
        X_mask[0, positions[1]] = 1
        Y = np.sum([X_num[0, positions[0]], X_num[0, positions[1]]])
        Y = np.round(Y, 2)

        X = np.append(X_num, X_mask, axis=0)
        print(X, Y, sep='\n')
        X_res.append(X.tolist())
        Y_res.append(Y.tolist())
        
    return X_res, Y_res

In [152]:
adding_problem_generator(2)

[[0.7  0.94 0.6  0.59 0.45 0.83 0.47 0.61]
 [0.   0.   1.   0.   0.   0.   1.   0.  ]]
1.07
[[0.64 0.82 0.37]
 [1.   1.   0.  ]]
1.46


([[[0.7, 0.94, 0.6, 0.59, 0.45, 0.83, 0.47, 0.61],
   [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]],
  [[0.64, 0.82, 0.37], [1.0, 1.0, 0.0]]],
 [1.07, 1.46])

In [168]:
ax=np.array(a, dtype=object)

In [174]:
ax[:,]

IndexError: index 2 is out of bounds for axis 1 with size 2

In [165]:
a.to_array()

AttributeError: 'list' object has no attribute 'to_array'

## Elman

In [ ]:
wih = np.random.randn(2, 16)

In [236]:
import numpy as np

# Define the Elman network RNN architecture
class ElmanNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        # Initialize weights for input-to-hidden and hidden-to-hidden connections
        self.w_ih = np.random.randn(input_size, hidden_size)
        self.w_hh = np.random.randn(hidden_size, hidden_size)
        self.b_h = np.zeros((hidden_size, 1))
        # Initialize weights for hidden-to-output connections
        self.w_ho = np.random.randn(hidden_size, output_size)
        self.b_o = np.zeros((output_size, 1))

    def forward(self, x):
        # Initialize hidden state with zeros
        h = np.zeros((x.shape[0], self.w_hh.shape[0]))
        # Initialize list to store hidden states for each time step
        hidden_states = []
        # Iterate over input sequence and update hidden state and output
        for t in range(x.shape[1]):
            # Update hidden state using input and previous hidden state
            h = np.tanh(np.dot(self.w_ih, x[:,t,:]) + np.dot(self.w_hh, h) + self.b_h)
            # Store current hidden state
            hidden_states.append(h)
        # Compute output using current hidden state
        o = np.dot(self.w_ho, h) + self.b_o
        # Return final output and list of hidden states
        return o, hidden_states

    def backward(self, x, y_true, y_pred, hidden_states, learning_rate):
        # Initialize gradients with zeros
        d_w_ih = np.zeros_like(self.w_ih)
        d_w_hh = np.zeros_like(self.w_hh)
        d_b_h = np.zeros_like(self.b_h)
        d_w_ho = np.zeros_like(self.w_ho)
        d_b_o = np.zeros_like(self.b_o)
        # Initialize error gradient with zeros
        d_error = 0
        # Iterate over time steps in reverse order and compute gradients
        for t in reversed(range(x.shape[1])):
            # Compute error gradient for output layer
            d_output = y_true - y_pred
            d_error += np.sum(d_output ** 2)
            # Compute gradient for hidden-to-output weights and biases
            d_w_ho += np.dot(d_output, hidden_states[t].T)
            d_b_o += d_output
            # Compute gradient for hidden state
            d_hidden = np.dot(self.w_ho.T, d_output) + np.dot(self.w_hh.T, d_hidden_next)
            # Compute gradient for hidden-to-hidden weights and biases
            d_w_hh += np.dot(d_hidden * (1 - hidden_states[t] ** 2), hidden_states[t-1].T)
            d_b_h += d_hidden * (1 - hidden_states[t] ** 2)
            # Compute gradient for input-to-hidden weights and biases
            d_w_ih += np.dot(d_hidden * (1 - hidden_states[t] ** 2), x[:,t].T)
        # Update weights and biases using gradients and learning rate
        self.w_ih += learning_rate * d_w_ih
        self.w_hh += learning_rate * d_w_hh
        self.b_h += learning_rate * d_b_h
        self.w_ho += learning_rate * d_w_ho
        self.b_o += learning_rate * d_b_o
        # Return total error gradient
        return d_error

# Define parameters for the Elman network RNN


In [237]:
train_X, train_Y = data_gen(100)
val_X, val_Y = data_gen(400)
test_X, test_Y = data_gen(100)

In [238]:
# Define hyperparameters
hidden_size = 16
learning_rate = 0.1
num_epochs = 100
#batch_size = 16
#num_batches = int(len(train_X) / batch_size)

# Initialize Elman network RNN
rnn = ElmanNetwork(input_size=2, hidden_size=hidden_size, output_size=1, learning_rate=learning_rate)

# Train the network
for epoch in range(num_epochs):
    # Shuffle the training set
    #indices = np.random.permutation(len(train_X))
    #train_X = train_X[indices]
    #train_Y = train_Y[indices]

    # Initialize total loss for epoch
    epoch_loss = 0.0

    # Train on batches
    #for batch in range(num_batches):
    # Get batch inputs and targets
    #start = batch * batch_size
    #end = start + batch_size
    #batch_X = train_X[start:end]
    #batch_Y = train_Y[start:end]

    # Forward pass
#     y_pred = rnn.forward(batch_X)
    y_pred = rnn.forward(train_X)

    # Compute loss
#     loss = mean_squared_error(batch_Y, y_pred)
    loss = mean_squared_error(train_Y, y_pred)
    epoch_loss += loss

    # Backward pass
#     rnn.backward(batch_X, batch_Y, y_pred)
    rnn.backward(train_X, train_Y, y_pred)
        
    # Compute average epoch loss
    #epoch_loss /= num_batches

    # Evaluate on validation set
    val_Y_pred = rnn.forward(val_X)
    val_loss = mean_squared_error(val_Y, val_Y_pred)

    # Print epoch results
    print("Epoch {}/{}: loss={:.4f}, val_loss={:.4f}".format(epoch+1, num_epochs, epoch_loss, val_loss))

    # Check stopping criterion
    if val_loss < 1e-4:
        print("Stopping criterion reached!")
        break

# Evaluate on test set
test_Y_pred = rnn.forward(test_X)
test_loss = mean_squared_error(test_Y, test_Y_pred)
print("Test loss: {:.4f}".format(test_loss))

ValueError: shapes (2,16) and (100,2) not aligned: 16 (dim 1) != 100 (dim 0)

In [220]:
import minpy.numpy as np
from minpy.nn import layers
from minpy.nn.model import ModelBase
from minpy.nn.solver import Solver
from minpy.nn.io import NDArrayIter
from examples.utils.data_utils import adding_problem_generator as data_gen


class RNNNet(ModelBase):
    def __init__(self,
                 batch_size=100,
                 input_size=2,  # input dimension
                 hidden_size=64,
                 num_classes=1):
        super(RNNNet, self).__init__()
        self.add_param(name='Wx', shape=(input_size, hidden_size))\
            .add_param(name='Wh', shape=(hidden_size, hidden_size))\
            .add_param(name='b', shape=(hidden_size,))\
            .add_param(name='Wa', shape=(hidden_size, num_classes))\
            .add_param(name='ba', shape=(num_classes,))

    def forward(self, X, mode):
        seq_len = X.shape[1]
        batch_size = X.shape[0]
        hidden_size = self.params['Wh'].shape[0]
        h = np.zeros((batch_size, hidden_size))
        for t in xrange(seq_len):
            h = layers.rnn_step(X[:, t, :], h, self.params['Wx'],
                                self.params['Wh'], self.params['b'])
        y = layers.affine(h, self.params['Wa'], self.params['ba'])
        return y

    def loss(self, predict, y):
        return layers.l2_loss(predict, y)


def main():
    model = RNNNet()
    x_train, y_train = data_gen(10000)
    x_test, y_test = data_gen(1000)

    train_dataiter = NDArrayIter(x_train,
                                 y_train,
                                 batch_size=100,
                                 shuffle=True)

    test_dataiter = NDArrayIter(x_test,
                                y_test,
                                batch_size=100,
                                shuffle=False)

    solver = Solver(model,
                    train_dataiter,
                    test_dataiter,
                    num_epochs=10,
                    init_rule='xavier',
                    update_rule='adam',
                    task_type='regression',
                    verbose=True,
                    print_every=20)
    solver.init()
    solver.train()


if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'minpy'